In [1]:
import json
import time
import csv
import re
import datetime
from enum import Enum, unique


import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [2]:
G_Debug = True   # if debug mode.  Debug = True. if Release mode, Debug = False.
G_Chrome_Hide = True
G_Debug_Pages = 1
G_Debug_Items = 10

G_FILE_NAME = 'tianjin'

In [3]:
#will bid
# sourceUrl = ("https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.60.53db26cdCGMLqt&category=50025969"  
#                    "&auction_source=0&province=%CC%EC%BD%F2&sorder=1&st_param=-1&auction_start_seg=-1")
# #done bid
sourceUrl = ("https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2"
               "&st_param=-1&auction_start_seg=&auction_start_from=2019-01-01&auction_start_to=2019-07-10&&spm=a213w.3064813.9001.2")




In [4]:
def log(page_id,item_id,status):
    print("page id: {0}, item id: {1}, status:{2}".format(page_id,item_id,status))

In [5]:
@unique
class Data(Enum):

    #release
    name = "name"
    year = "year"
    month = "month"
    start = "start"
    end = "end"
    times = "bidNumber"
    title = "title"
    origin_price = "originPrice"
    current_price = "currentPrice"
    discount = "discount"
    unit = "unit"
    bid_count = "bidCount"
    delay_count = "delayCount"
    apply_count = "applyCount"    
    area = "area"
    item_url = "itemUrl"
    
    #debug
    areaA = "areaA"
    areaB = "areaB"
    bid_id = "id"
    status = "status"
    consult_price = "consultPrice"
    market_price = "marketPrice"

In [6]:
class DataManagement:
    def __init__(self,file_name):
        self.data = {}

        self.file_name = file_name
    
        self.head = []
        for data in Data:
            self.head.append(data.value)
        
        self.__open_file()

    def __process_attri_name(self,attribute, data):
        index = data.find("通过")
        data = data[4:index]
        self.data[attribute] =  data     
            
    def __process_attri_area(self,attribute, data):
        left = data.find("建筑面积")
        if(left != -1):
            size = data[(left+4) : (left+22)]
            areas = re.findall(r"\d+\.\d*",size)
            self.data[attribute] = areas[0] if areas else 0
        else:
            self.data[attribute] = 0
        
    def __process_attri_date(self,attribute, data):
        time1 = int(int(data)/1000 )
        self.data[attribute] = str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time1)))
        
    def __process_attri_times(self,attribute, data):
        keyword_postion = data.find("卖")
        if(keyword_postion != -1):
            data = data[1:keyword_postion+1]
        else:
            data = "no content"
            
        self.data[attribute] = data



    def __process_attri_common(self,attribute, data):
        self.data[attribute] =  data        
     
    def __process_attri_url(self,attribute, data):
        url = "https:"+data
        self.data[attribute] =  url        

    
    def __clac_data(self):
        # 
        if (int(self.data[Data.consult_price.value]) > 0):
            originPrice = self.data[Data.consult_price.value]
        else:
            originPrice = self.data[Data.market_price.value]
        #
        if (int(originPrice)  == 0):
            self.data[Data.discount.value] = "error"
        else:
            self.data[Data.discount.value]  = float("%.2f" % float(int(self.data[Data.consult_price.value])/originPrice))
 
       # change unit to from RMB yuan to  RMB Wan
        self.data[Data.origin_price.value]  = float("%.2f" % float(int(originPrice)/10000))        
        self.data[Data.current_price.value] = float("%.2f" % float(int(self.data[Data.current_price.value])/10000))

      # set area
        if(float(self.data[Data.areaA.value]) > 0):
            self.data[Data.area.value]  = self.data[Data.areaA.value]
            self.data[Data.unit.value] = float("%.2f" % float(int(self.data[Data.current_price.value])/float(self.data["area"]) ))
        elif(float(self.data[Data.areaB.value]) > 0):
            self.data[Data.area.value] = self.data[Data.areaB.value]
            self.data[Data.unit.value] = float("%.2f" % float(int(self.data[Data.current_price.value])/float(self.data["area"]) ))
        else:
            self.data[Data.area.value] = 0
            self.data[Data.unit.value] = 0
      
        ## set year, month
        date_start = self.data[Data.start.value]
        data_split = date_start.split("-",2)
        self.data[Data.year.value] = data_split[0]
        self.data[Data.month.value] = data_split[1]

    
    def set_data(self,attribute, data):
        if(attribute == Data.name.value):
            self.__process_attri_name(attribute, data)
        elif(attribute == Data.areaA.value or attribute == Data.areaB.value):
            self.__process_attri_area(attribute, data)
        elif(attribute == Data.start.value or attribute == Data.end.value):
            self.__process_attri_date(attribute, data)
        elif(attribute == Data.times.value):
            self.__process_attri_times(attribute, data)
        elif(attribute == Data.item_url.value):
            self.__process_attri_url(attribute, data)
        else:
            self.__process_attri_common(attribute,data)    
    
    
    def get_data(self,attribute):
        return self.data[attribute]
    
    def __open_file(self):
        self.csv_file = open(self.file_name+".csv","w+")
        self.writer = csv.writer(self.csv_file)        
        self.writer.writerow(self.head)

    def write_file(self):
        
        self.__clac_data()
        
        row = tuple(self.data[d.value] for d in Data)
        
        self.writer.writerow(row)
    
    def close_file(self):
        self.csv_file.close()
        csv = pd.read_csv(self.file_name+".csv", encoding='utf-8')
        
        #debug excel
        csv.to_excel(self.file_name+"_debug.xlsx", sheet_name='data')
        
        #realse excel
        relase_column = list(self.head)
        relase_column.remove(Data.areaA.value)
        relase_column.remove(Data.areaB.value)
        relase_column.remove(Data.bid_id.value)
        relase_column.remove(Data.status.value)
        relase_column.remove(Data.consult_price.value)
        relase_column.remove(Data.market_price.value)
        
        csv.to_excel(self.file_name+"_release.xlsx", sheet_name='data',columns =relase_column )
            


In [7]:
class SfSpider:
    
    
    def __init__(self,debug,debug_pages,debug_items,head_less,url,file_name):
        
        self.debug = debug
        self.debug_pages = debug_pages
        self.debug_items = debug_items
        self.url = url

        
        
        self.data = DataManagement(file_name)
        
        if(head_less):
            #set headless
            chrome_options=Options()
            chrome_options.add_argument('--headless')
            self.driver = webdriver.Chrome(options=chrome_options)
        else:
            self.driver = webdriver.Chrome()
            

        # Setup wait for later
        self.wait = WebDriverWait(self.driver, 10)

        
    def __wait_and_click(self,mode,value,sleep_time):
    #  Sometimes click fails unreasonably. So tries to click at all cost.
        try:
            if(mode == "LINK_TEXT"):
                elem = self.wait.until(EC.element_to_be_clickable((By.LINK_TEXT, value)))
            elif(mode == "CSS_SELECTOR"):
                elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,value)))
            elif(mode == "XPATH"):
                elem = self.wait.until(EC.element_to_be_clickable((By.XPATH,value)))
            else:
                print('wait_and_click() error...')
                
            elem.click()
            time.sleep(sleep_time)
        except Exception as e:
            print('Click time out - {}'.format(value))
            print('Refreshing browser...')
            self.driver.refresh()
            time.sleep(2)
            return self.__wait_and_click(mode,value,sleep_time)

        return elem
        
        
    def do_crawling(self):
        
        self.driver.get(self.url)
        
        assert len(self.driver.window_handles) == 1

        elem = self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))
        
        if(self.debug):
            pages = self.debug_pages
        else:
            pages = int(self.driver.find_element_by_class_name("page-total").text)
                    
        for page in range(1,pages+1):
            self.__do_page_crawling(page)

        self.driver.quit()
        self.data.close_file()
        
        

    def __do_page_crawling(self,page):
        if(page > 1):
            self.__wait_and_click("LINK_TEXT",str(page),1)

        self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
        jsonData =  json.loads(self.driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML'))
        page_data = jsonData["data"]

        if(self.debug):
            items = self.debug_items
        else:
            items = len(page_data)    

        for item in range(items):
            item_data = page_data[item]
            log(page,item,item_data["status"])
            
            self.data.set_data(Data.bid_id.value,item_data[Data.bid_id.value])
            self.data.set_data(Data.status.value,item_data[Data.status.value])
            self.data.set_data(Data.start.value,item_data[Data.start.value])
            self.data.set_data(Data.end.value,item_data[Data.end.value])
            self.data.set_data(Data.title.value,item_data[Data.title.value])
            self.data.set_data(Data.consult_price.value,item_data[Data.consult_price.value])
            self.data.set_data(Data.market_price.value,item_data[Data.market_price.value])
            self.data.set_data(Data.current_price.value,item_data[Data.current_price.value])            
            self.data.set_data(Data.bid_count.value,item_data[Data.bid_count.value])
            self.data.set_data(Data.delay_count.value,item_data[Data.delay_count.value])
            self.data.set_data(Data.apply_count.value,item_data[Data.apply_count.value])
            self.data.set_data(Data.item_url.value,item_data[Data.item_url.value])


            self.__do_item_crawling(page,item)


    def __do_item_crawling(self,page_id,item_id):

        self.__wait_and_click("CSS_SELECTOR","#pai-item-"+str(self.data.get_data(Data.bid_id.value)),1)

        # Wait for the new window or tab
        self.wait.until(EC.number_of_windows_to_be(2))
        handles = self.driver.window_handles
        self.driver.switch_to.window(handles[1])

       # bid times
        self.wait.until(EC.presence_of_element_located((By.XPATH, "/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]")))
        self.data.set_data(Data.times.value,self.driver.find_element_by_xpath("/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]").get_attribute('textContent'))
        #TODO: degfine new field

       # 变卖公告，竞买公告
        self.__wait_and_click("CSS_SELECTOR","#J_DetailTabMenu > li.current.first > a",0.5)

        self.data.set_data(Data.areaA.value,self.driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent'))


        self.__wait_and_click("LINK_TEXT","标的物介绍",0.5)

        self.data.set_data(Data.areaB.value,self.driver.find_element_by_id("J_desc").get_attribute('textContent'))


        if(self.data.get_data(Data.status.value) == "done"):

            self.__wait_and_click("LINK_TEXT","竞价成功确认书",0.5)

            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))

            name = self.driver.find_element_by_class_name("c-content").text
        elif(self.data.get_data(Data.status.value) == "failure"):
            name = "用户姓名流拍通过"
        else:
            name = "用户姓名即将开始通过"
        self.data.set_data(Data.name.value,name)

        self.data.write_file()
        self.driver.close()
        self.driver.switch_to.window(handles[0])

In [8]:
if __name__ == '__main__':

    start = datetime.datetime.now()
    print("start : {0}".format(start))
    
    sf = SfSpider(G_Debug,G_Debug_Pages,G_Debug_Items,G_Chrome_Hide,sourceUrl,G_FILE_NAME)
    
    sf.do_crawling()
    
    end = datetime.datetime.now()
    print("end : {0}".format(end))
    print("duration: {0}".format(end-start))



start : 2019-07-12 08:11:10.760870
page id: 1, item id: 0, status:done
page id: 1, item id: 1, status:done
page id: 1, item id: 2, status:done
page id: 1, item id: 3, status:failure
page id: 1, item id: 4, status:failure
page id: 1, item id: 5, status:failure
page id: 1, item id: 6, status:failure
page id: 1, item id: 7, status:failure
page id: 1, item id: 8, status:failure
page id: 1, item id: 9, status:failure
end : 2019-07-12 08:11:45.855429
duration: 0:00:35.094559
